In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Analyzing ACRONYM data

In [46]:
import pandas as pd
import numpy as np

# path = '/data/manifolds/acronym/bullet_grasps/fullrun_grasps.csv'
path = '/data/manifolds/acronym/bullet_grasps/grasps.csv'

df = pd.read_csv(path)
# print(df)
both_true = df.query('flex_success==1 & pybullet_success==1')
print("both true", len(both_true))
flex_true = df.query('flex_success==1')
print("flex true", len(flex_true))
bullet_true = df.query('pybullet_success==1')
print("bullet true", len(bullet_true))

print(f"true bullet positives over flex positives: {len(both_true) / len(flex_true):.3f}")
print(f"true bullet positives over all positives: {len(both_true) / len(bullet_true):.3f}")

both_false = df.query('flex_success==0 & pybullet_success==0')
print("both false", len(both_false))
flex_false = df.query('flex_success==0')
print("flex false", len(flex_false))
bullet_false = df.query('pybullet_success==0')
print("bullet false", len(bullet_false))

print(f"true bullet negatives over flex negatives: {len(both_false) / len(flex_false):.3f}")
print(f"true bullet negatives over all negatives: {len(both_false) / len(bullet_false):.3f}")



both true 240
flex true 1139
bullet true 247
true bullet positives over flex positives: 0.211
true bullet positives over all positives: 0.972
both false 854
flex false 861
bullet false 1753
true bullet negatives over flex negatives: 0.992
true bullet negatives over all negatives: 0.487


In [47]:
print("Error reason")
print(df.loc[:, 'err_info'].value_counts())

Error reason
grasp     1740
linear      13
Name: err_info, dtype: int64


# Analyzing planning trials

In [94]:
# 100 scenes, (30 grasps each scene (we are not following this strictly, check the paper for details)), 10 runs

def get_exp_paths(base_path):
    method_name = base_path.split('/')[-1]
    exp_suffixes = [f'{method_name}_Fixed', f'{method_name}_Proj', f'{method_name}_OMG']
    exp_paths = [[], [], []]
    for path in os.listdir(base_path):
        for i, exp_suffix in enumerate(exp_suffixes):
            if exp_suffix not in path:
                continue
#             print(path)
            scene_paths = os.listdir(f'{base_path}/{path}')
            if len(scene_paths) < 100:
                print(f"Warning: skipping {path} as there are only {len(scene_paths)} scenes")
                continue
            exp_paths[i].append(f'{base_path}/{path}')
            break
    
    # Check that all experiments have 10 runs
    for i, exp_list in enumerate(exp_paths):
        if len(exp_list) != 10:
            print(f"Warning: {len(exp_list)} total runs for {exp_suffixes[i]}")
    return exp_paths
            
def get_method_results(base_path):
    """
    Returns evaluation metrics for a grasp inference method 
    Run on 10 runs of 100 scenes. 
    Evaluation metrics: execution success, planning success, smoothness, collision, time
    Returns a pandas dataframe with each eval metric in its own column
    """
    exp_paths = get_exp_paths(base_path)
    runs_metrics = {
        'method': [],
        'exec': [],
        'plan': [],
        'smth': [],
        'coll': [],
        'plan_time': [],
        'infer_time': []
    }
#   TODO load data.npy, collect metrics, return data frame
    for exp_list in exp_paths:
        for path in exp_list:
            scene_paths = os.listdir(f'{path}')
            
            exp_metrics = {
                'method': [],
                'exec': [],
                'plan': [],
                'smth': [],
                'coll': [],
                'plan_time': [],
                'infer_time': []
            }
            for i, scene_path in enumerate(scene_paths):
                try:
                    data = np.load(f'{path}/{scene_path}/data.npy', allow_pickle=True, encoding='latin1')
                except Exception as e:
                    print(e)
                    print(f"Stopping early at {i}")
                    break
                rew, info, plan, infer_time = data
                if len(info) < 1:
                    continue
                
                method = '_'.join(path.split('/')[-1].split('_')[1:])
                exp_metrics['method'].append(method)
                exp_metrics['exec'].append(rew)
                exp_metrics['plan'].append(1 if info[-1]['terminate'] else 0)
                exp_metrics['smth'].append(info[-1]['smooth'])
                exp_metrics['coll'].append(info[-1]['obs'])
                exp_metrics['plan_time'].append(info[-1]['time'])
                exp_metrics['infer_time'].append(infer_time)
#                 else:
#                     exp_metrics['plan'].append(0)
#                     exp_metrics['smth'].append(None)
#                     exp_metrics['coll'].append(None)
#                     exp_metrics['time'].append(None)
            runs_metrics['method'].append(exp_metrics['method'][0])
            runs_metrics['exec'].append(np.mean(exp_metrics['exec']))
            runs_metrics['plan'].append(np.mean(exp_metrics['plan']))
            runs_metrics['smth'].append(np.mean(exp_metrics['smth']))
            runs_metrics['coll'].append(np.mean(exp_metrics['coll']))
            runs_metrics['plan_time'].append(np.mean(exp_metrics['plan_time']))
            runs_metrics['infer_time'].append(np.mean(exp_metrics['infer_time']))
#         break
    m_df = pd.DataFrame(runs_metrics)
    print(m_df.round(3))
    
    return m_df
            
acronym_runpath = '/checkpoint/thomasweng/grasp_manifolds/pybullet_eval/acronym'
acronym_df = get_method_results(acronym_runpath)
contactgraspnet_runpath = '/checkpoint/thomasweng/grasp_manifolds/pybullet_eval/contact_graspnet'
contactgraspnet_df = get_method_results(contactgraspnet_runpath)

           method  exec  plan    smth    coll  plan_time  infer_time
0   acronym_Fixed  0.95  0.98  12.526  10.576      0.041         0.0
1   acronym_Fixed  0.91  0.99  12.917  12.846      0.040         0.0
2   acronym_Fixed  0.93  0.98  12.538  10.344      0.048         0.0
3   acronym_Fixed  0.91  0.99  13.477  13.936      0.034         0.0
4   acronym_Fixed  0.95  0.99  12.275  10.353      0.035         0.0
5   acronym_Fixed  0.92  0.96  13.065  12.418      0.073         0.0
6   acronym_Fixed  0.93  0.98  12.000   9.624      0.040         0.0
7   acronym_Fixed  0.95  0.95  12.643  11.513      0.059         0.0
8   acronym_Fixed  0.94  0.98  13.162  14.835      0.041         0.0
9   acronym_Fixed  0.92  0.97  13.729  10.035      0.053         0.0
10   acronym_Proj  0.89  0.97   7.904   9.887      0.051         0.0
11   acronym_Proj  0.96  0.99   8.038   9.972      0.036         0.0
12   acronym_Proj  0.91  0.99   8.115  11.216      0.039         0.0
13   acronym_Proj  0.92  0.98   7.

In [96]:
def get_mean_std_str(df):
    means = list(df[['exec', 'plan', 'smth', 'coll', 'plan_time', 'infer_time']].mean().values.round(3))
    stds = list(df[['exec', 'plan', 'smth', 'coll', 'plan_time', 'infer_time']].std().values.round(3))
    return [f"{m}+{s}" for m, s in zip(means, stds)]

df = pd.DataFrame(columns=acronym_df.columns)
df.loc[0] = ['acronym_Fixed'] + get_mean_std_str(acronym_df.loc[acronym_df['method'] == "acronym_Fixed"])
df.loc[1] = ['acronym_Proj'] + get_mean_std_str(acronym_df.loc[acronym_df['method'] == "acronym_Proj"])
df.loc[2] = ['acronym_OMG'] + get_mean_std_str(acronym_df.loc[acronym_df['method'] == "acronym_OMG"])
df.loc[3] = ['contact_graspnet_Fixed'] + get_mean_std_str(contactgraspnet_df.loc[contactgraspnet_df['method'] == "contact_graspnet_Fixed"])
df.loc[4] = ['contact_graspnet_Proj'] + get_mean_std_str(contactgraspnet_df.loc[contactgraspnet_df['method'] == "contact_graspnet_Proj"])
df.loc[5] = ['contact_graspnet_OMG'] + get_mean_std_str(contactgraspnet_df.loc[contactgraspnet_df['method'] == "contact_graspnet_OMG"])

df

,method,exec,plan,smth,coll,plan_time,infer_time
0,acronym_Fixed,0.931+0.016,0.977+0.013,12.833+0.539,11.648+1.785,0.046+0.012,0.0+0.0
1,acronym_Proj,0.914+0.023,0.98+0.013,7.957+0.134,9.785+1.077,0.042+0.009,0.0+0.0
2,acronym_OMG,0.926+0.025,0.97+0.017,13.354+0.384,12.639+2.033,0.176+0.028,0.0+0.0
3,contact_graspnet_Fixed,0.295+0.032,0.981+0.01,13.973+0.282,4.862+1.033,0.041+0.009,0.319+0.003
4,contact_graspnet_Proj,0.34+0.026,0.989+0.015,8.384+0.21,6.613+2.213,0.035+0.008,0.325+0.001
5,contact_graspnet_OMG,0.372+0.042,0.981+0.016,12.975+0.704,4.917+1.713,0.118+0.021,0.326+0.001


In [ ]:
exp_suffix = 'acronym_Fixed'

out_path = '/checkpoint/thomasweng/grasp_manifolds/pybullet_eval/acronym'

exp_paths = []
for exp_path in os.listdir(out_path):
#     if 'dbg' in date_path or 'old_runs' in date_path:
#         continue
#     for exp_path in os.listdir(f'{out_path}/{date_path}'):
    if exp_suffix in exp_path:
        exp_paths.append(f'{out_path}/{exp_path}')

if len(exp_paths) != 10:
    print(f"Warning: {len(exp_paths)} total runs for {exp_suffix}: {len(exp_paths)}")
else:
    print("10 runs")

runs_metrics = {
    'exec': [],
    'plan': [],
    'smth': [],
    'coll': [],
    'time': []
}
for exp_path in exp_paths:
    scene_paths = sorted(os.listdir(exp_path), key=lambda x: int(x.split('_')[1]))
    if len(scene_paths) < 100:
        print(f"Skipping {exp_path}")
        continue

    exp_metrics = {
        'exec': [],
        'plan': [],
        'smth': [],
        'coll': [],
        'time': [],
    }
    for i, scene_path in enumerate(scene_paths):
        try:
            data = np.load(f'{exp_path}/{scene_path}/data.npy', allow_pickle=True, encoding='latin1')
        except Exception as e:
            print(e)
            print(f"Stopping early at {i}")
            break
#         print(data)
        rew, info, plan, inference_duration = data
        exp_metrics['exec'].append(rew)
        exp_metrics['plan'].append(1 if info[-1]['terminate'] else 0)
        exp_metrics['smth'].append(info[-1]['smooth'])
        exp_metrics['coll'].append(info[-1]['obs'])
        exp_metrics['time'].append(info[-1]['time'])
#     print(exp_metrics['time'])
    runs_metrics['exec'].append(np.mean(exp_metrics['exec']))
    runs_metrics['plan'].append(np.mean(exp_metrics['plan']))
    runs_metrics['smth'].append(np.mean(exp_metrics['smth']))
    runs_metrics['coll'].append(np.mean(exp_metrics['coll']))
    runs_metrics['time'].append(np.mean(exp_metrics['time']))
mean_exec = np.mean(runs_metrics['exec'])
mean_plan = np.mean(runs_metrics['plan'])
mean_smth = np.mean(runs_metrics['smth'])
mean_coll = np.mean(runs_metrics['coll'])
mean_time = np.mean(runs_metrics['time'])
print(f'{mean_exec:.2f}+/-{np.std(runs_metrics["exec"]):.2f}')
print(f'{mean_plan:.2f}+/-{np.std(runs_metrics["plan"]):.2f}')
print(f'{mean_smth:.2f}+/-{np.std(runs_metrics["smth"]):.2f}')
print(f'{mean_coll:.2f}+/-{np.std(runs_metrics["coll"]):.2f}')
print(f'{mean_time:.2f}+/-{np.std(runs_metrics["time"]):.2f}')